# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [43]:
import pandas as pd

In [44]:
data = pd.read_csv('census_data.csv')

In [45]:
data.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [46]:
data['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [47]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [48]:
data['income_bracket'] = data['income_bracket'].apply(label_fix)

In [49]:
data = data.dropna(axis = 0)

In [50]:
cols_to_norm = ['age', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']


In [51]:
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

### Perform a Train Test Split on the Data

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X = data.drop(['income_bracket'], axis = 1)
Y = data['income_bracket']

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [55]:
data.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

** Import Tensorflow **

In [56]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [57]:
workclass = tf.feature_column.categorical_column_with_hash_bucket('workclass', hash_bucket_size = data['workclass'].unique().size)
education = tf.feature_column.categorical_column_with_hash_bucket('education', hash_bucket_size = data['education'].unique().size)
marital_status = tf.feature_column.categorical_column_with_hash_bucket('marital_status', hash_bucket_size = data['marital_status'].unique().size)
occupation = tf.feature_column.categorical_column_with_hash_bucket('occupation', hash_bucket_size = data['occupation'].unique().size)
relationship = tf.feature_column.categorical_column_with_hash_bucket('relationship', hash_bucket_size = data['relationship'].unique().size)
race = tf.feature_column.categorical_column_with_hash_bucket('race', hash_bucket_size = data['race'].unique().size)
gender = tf.feature_column.categorical_column_with_vocabulary_list('gender', ['Female', 'Male'])
native_country = tf.feature_column.categorical_column_with_hash_bucket('native_country', hash_bucket_size = data['native_country'].unique().size)


** Create the continuous feature_columns for the continuous values using numeric_column **

In [58]:
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education_num')
capital_gain = tf.feature_column.numeric_column('capital_gain')
capital_loss = tf.feature_column.numeric_column('capital_loss')
hours_week = tf.feature_column.numeric_column('hours_per_week')

** Put all these variables into a single list with the variable name feat_cols **

In [59]:
feat_cols = [age, workclass, education, education_num, marital_status, occupation, relationship, race, gender, capital_gain, capital_loss, hours_week, native_country]

In [60]:
feat_cols

[_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _HashedCategoricalColumn(key='workclass', hash_bucket_size=9, dtype=tf.string),
 _HashedCategoricalColumn(key='education', hash_bucket_size=16, dtype=tf.string),
 _NumericColumn(key='education_num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _HashedCategoricalColumn(key='marital_status', hash_bucket_size=7, dtype=tf.string),
 _HashedCategoricalColumn(key='occupation', hash_bucket_size=15, dtype=tf.string),
 _HashedCategoricalColumn(key='relationship', hash_bucket_size=6, dtype=tf.string),
 _HashedCategoricalColumn(key='race', hash_bucket_size=5, dtype=tf.string),
 _VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0),
 _NumericColumn(key='capital_gain', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='capital_loss', shape=(1,), default_

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [61]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=Y_train, batch_size=10, num_epochs=1000, shuffle=True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [62]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\4CFA~1\\AppData\\Local\\Temp\\tmpn_d7zw80', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000003A90BEB080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [63]:
model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\4CFA~1\AppData\Local\Temp\tmpn_d7zw80\model.ckpt.
INFO:tensorflow:loss = 6.931472, step = 1
INFO:tensorflow:global_step/sec: 133.317
INFO:tensorflow:loss = 4.4819045, step = 101 (0.754 sec)
INFO:tensorflow:global_step/sec: 309.44
INFO:tensorflow:loss = 3.2601838, step = 201 (0.319 sec)
INFO:tensorflow:global_step/sec: 301.716
INFO:tensorflow:loss = 3.1889381, step = 301 (0.338 sec)
INFO:tensorflow:global_step/sec: 307.039
INFO:tensorflow:loss = 3.1431003, step = 401 (0.325 sec)
INFO:tensorflow:global_step/sec: 299.325
INFO:tensorflow:loss = 1.8840094, step = 501 (0.334 sec)
INFO:tensorflow:global_step/sec: 282.796
INFO:tensorflow:loss = 8.5540285, step = 601 (0.349 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:loss = 3.2419906, step = 8001 (0.364 sec)
INFO:tensorflow:global_step/sec: 274.208
INFO:tensorflow:loss = 8.140498, step = 8101 (0.365 sec)
INFO:tensorflow:global_step/sec: 285.712
INFO:tensorflow:loss = 4.7900696, step = 8201 (0.355 sec)
INFO:tensorflow:global_step/sec: 279.177
INFO:tensorflow:loss = 3.7475827, step = 8301 (0.353 sec)
INFO:tensorflow:global_step/sec: 283.639
INFO:tensorflow:loss = 3.4648356, step = 8401 (0.355 sec)
INFO:tensorflow:global_step/sec: 282.26
INFO:tensorflow:loss = 3.759623, step = 8501 (0.357 sec)
INFO:tensorflow:global_step/sec: 289.913
INFO:tensorflow:loss = 2.0553997, step = 8601 (0.340 sec)
INFO:tensorflow:global_step/sec: 277.921
INFO:tensorflow:loss = 3.8970118, step = 8701 (0.360 sec)
INFO:tensorflow:global_step/sec: 280.038
INFO:tensorflow:loss = 7.6958656, step = 8801 (0.356 sec)
INFO:tensorflow:global_step/sec: 278.301
INFO:tensorflow:loss = 5.511405, step = 8901 (0.359 sec)
INFO:tensorflow:global_step/sec: 281.445
INFO:tensorflo

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [64]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, y=Y_test, batch_size=10, num_epochs=1, shuffle=False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [65]:
predictions = model.predict(eval_input_func)

In [66]:
predicts = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\4CFA~1\AppData\Local\Temp\tmpn_d7zw80\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


** Each item in your list will look like this: **

In [67]:
for pred in predicts:
    print(pred)

{'logits': array([-1.1149942], dtype=float32), 'logistic': array([0.24694102], dtype=float32), 'probabilities': array([0.75305897, 0.24694099], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.6458511], dtype=float32), 'logistic': array([0.02543535], dtype=float32), 'probabilities': array([0.9745646 , 0.02543534], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.0821493], dtype=float32), 'logistic': array([0.2530995], dtype=float32), 'probabilities': array([0.7469005, 0.2530995], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.2329445], dtype=float32), 'logistic': array([0.03794461], dtype=float32), 'probabilities': array([0.9620554 , 0.03794461], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.6824363], dtype=float32), 'log

{'logits': array([-1.6104268], dtype=float32), 'logistic': array([0.16652937], dtype=float32), 'probabilities': array([0.83347064, 0.16652937], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.2234404], dtype=float32), 'logistic': array([0.09766519], dtype=float32), 'probabilities': array([0.9023348 , 0.09766519], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.3306308], dtype=float32), 'logistic': array([0.0886177], dtype=float32), 'probabilities': array([0.9113823 , 0.08861771], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.9859495], dtype=float32), 'logistic': array([0.01823607], dtype=float32), 'probabilities': array([0.9817639 , 0.01823607], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.7933173], dtype=float32), 'l

{'logits': array([-3.6984456], dtype=float32), 'logistic': array([0.02416365], dtype=float32), 'probabilities': array([0.97583634, 0.02416365], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-4.1614857], dtype=float32), 'logistic': array([0.01534524], dtype=float32), 'probabilities': array([0.9846548 , 0.01534524], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([0.28288996], dtype=float32), 'logistic': array([0.57025456], dtype=float32), 'probabilities': array([0.4297454 , 0.57025456], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-3.9841938], dtype=float32), 'logistic': array([0.01826753], dtype=float32), 'probabilities': array([0.9817324 , 0.01826753], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.0016706], dtype=float32), '

{'logits': array([-1.79393], dtype=float32), 'logistic': array([0.14259157], dtype=float32), 'probabilities': array([0.8574084 , 0.14259157], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.6480994], dtype=float32), 'logistic': array([0.02537967], dtype=float32), 'probabilities': array([0.97462034, 0.02537967], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.3340213], dtype=float32), 'logistic': array([0.08834425], dtype=float32), 'probabilities': array([0.9116558 , 0.08834425], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.165991], dtype=float32), 'logistic': array([0.23758042], dtype=float32), 'probabilities': array([0.76241964, 0.23758042], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([0.08408737], dtype=float32), 'log

{'logits': array([-2.0281675], dtype=float32), 'logistic': array([0.11627709], dtype=float32), 'probabilities': array([0.8837229 , 0.11627709], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.8343265], dtype=float32), 'logistic': array([0.13772367], dtype=float32), 'probabilities': array([0.8622763 , 0.13772367], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([0.395679], dtype=float32), 'logistic': array([0.59764904], dtype=float32), 'probabilities': array([0.40235093, 0.59764904], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([1.2266474], dtype=float32), 'logistic': array([0.77323127], dtype=float32), 'probabilities': array([0.22676875, 0.77323127], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-1.0324619], dtype=float32), 'log

...

{'logits': array([-2.720447], dtype=float32), 'logistic': array([0.06177755], dtype=float32), 'probabilities': array([0.9382225 , 0.06177755], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.1747042], dtype=float32), 'logistic': array([0.23600572], dtype=float32), 'probabilities': array([0.7639943 , 0.23600574], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.57345784], dtype=float32), 'logistic': array([0.36043933], dtype=float32), 'probabilities': array([0.63956064, 0.3604393 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.169929], dtype=float32), 'logistic': array([0.10248356], dtype=float32), 'probabilities': array([0.8975165 , 0.10248357], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.99399805], dtype=float32), '

{'logits': array([-3.4859252], dtype=float32), 'logistic': array([0.02971537], dtype=float32), 'probabilities': array([0.9702846 , 0.02971536], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.7870238], dtype=float32), 'logistic': array([0.05802943], dtype=float32), 'probabilities': array([0.9419706 , 0.05802943], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.471936], dtype=float32), 'logistic': array([0.18664853], dtype=float32), 'probabilities': array([0.8133515 , 0.18664855], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-5.229873], dtype=float32), 'logistic': array([0.00532569], dtype=float32), 'probabilities': array([0.9946743 , 0.00532569], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.5976003], dtype=float32), 'lo

{'logits': array([-3.438388], dtype=float32), 'logistic': array([0.03111705], dtype=float32), 'probabilities': array([0.968883  , 0.03111705], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.3804739], dtype=float32), 'logistic': array([0.08467383], dtype=float32), 'probabilities': array([0.9153261 , 0.08467383], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.7146642], dtype=float32), 'logistic': array([0.15255973], dtype=float32), 'probabilities': array([0.8474403 , 0.15255973], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.7061162], dtype=float32), 'logistic': array([0.33045757], dtype=float32), 'probabilities': array([0.66954243, 0.3304576 ], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-3.4801147], dtype=float32), 'l

{'logits': array([-3.112748], dtype=float32), 'logistic': array([0.04258447], dtype=float32), 'probabilities': array([0.9574156 , 0.04258447], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([0.00715899], dtype=float32), 'logistic': array([0.50178975], dtype=float32), 'probabilities': array([0.49821028, 0.50178975], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([0.9589908], dtype=float32), 'logistic': array([0.72291964], dtype=float32), 'probabilities': array([0.27708027, 0.72291964], dtype=float32), 'class_ids': array([1], dtype=int64), 'classes': array([b'1'], dtype=object)}
{'logits': array([-0.79311776], dtype=float32), 'logistic': array([0.31149963], dtype=float32), 'probabilities': array([0.68850034, 0.31149963], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-2.080829], dtype=float32), 'lo

{'logits': array([-2.6382523], dtype=float32), 'logistic': array([0.06671678], dtype=float32), 'probabilities': array([0.9332832 , 0.06671678], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.2808793], dtype=float32), 'logistic': array([0.2174006], dtype=float32), 'probabilities': array([0.7825994 , 0.21740058], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-0.0990572], dtype=float32), 'logistic': array([0.47525597], dtype=float32), 'probabilities': array([0.5247441 , 0.47525597], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([-1.4437022], dtype=float32), 'logistic': array([0.1909727], dtype=float32), 'probabilities': array([0.80902725, 0.19097267], dtype=float32), 'class_ids': array([0], dtype=int64), 'classes': array([b'0'], dtype=object)}
{'logits': array([12.020106], dtype=float32), 'log

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [68]:
class_ids = []
for pred in predicts:
    class_ids.append(pred['class_ids'][0])

In [69]:
class_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [70]:
from sklearn import metrics

In [71]:
Y = class_ids
class_names = ['0', '1']
print(metrics.classification_report(Y_test, Y, target_names=class_names))

             precision    recall  f1-score   support

          0       0.84      0.95      0.89      7436
          1       0.74      0.44      0.55      2333

avg / total       0.82      0.83      0.81      9769



# Great Job!